In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from tensorflow.keras import models, layers

In [5]:
laps = pd.read_csv("../raw_data/laps.csv")
track_status = pd.read_csv("../raw_data/track_status.csv")
weather = pd.read_csv("../raw_data/weather.csv")

/var/folders/f9/4fv5w4w54kggnjcsztxt19wm0000gn/T/ipykernel_47724/2337435297.py:1: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  laps = pd.read_csv("../raw_data/laps.csv")


In [6]:
def add_race_progress(df):
    # Group data to get lap number per year per race
    grouped_data = df.groupby(by = ["Year", "Location"], as_index=False)["LapNumber"].max().rename(columns={"LapNumber":"TotalLaps"})
    grouped_data["Year_Location"] = grouped_data["Year"].map(str) + grouped_data["Location"]
    # Group data to get same info as grouped_data
    df["Year_Location"] = df["Year"].map(str) + df["Location"]
    # Merge data
    df = df.merge(grouped_data, on="Year_Location")
    # Clean data frame
    df.drop(columns=["Year_Location", "Year_y", "Location_y"], inplace=True)
    df["RaceProgress"] = df["LapNumber"] / df["TotalLaps"]
    df.rename(columns={"Location_x": "Location", "Year_x": "Year"}, inplace=True)
    return df

In [7]:
def mask_race_percentage(df, percentage):
    df = df[df["RaceProgress"] > percentage]
    return df

In [8]:
def get_tyre_stress_level(df):
    params = {'Mexico City': 4,
    'Lusail': 1,
    'Shanghai': 2,
    'Mugello': 1,
    'Hockenheim': 3,
    'Jeddah': 3,
    'Imola': 3,
    'São Paulo': 3,
    'Montréal': 3,
    'Singapore': 4,
    'Marina Bay': 4,
    'Barcelona': 1,
    'Spain': 1,
    'Spa-Francorchamps': 1,
    'Spielberg': 3,
    'Melbourne': 3,
    'Budapest': 3,
    'Nürburgring': 3,
    'Suzuka': 1,
    'Austin': 1,
    'Zandvoort': 1,
    'Bahrain': 3,
    'Portimão': 2,
    'Sochi': 4,
    'Monza': 1,
    'Yas Marina': 3,
    'Monaco': 5,
    'Miami': 3,
    'Istanbul': 1,
    'Baku': 3,
    'Monte Carlo': 5,
    'Yas Island': 3,
    'Le Castellet': 2,
    'Sakhir': 3,
    'Silverstone': 1}
    df["TyreStressLevel"] = df["Location"].map(params)
    return df


In [15]:
laps = add_race_progress(laps)
laps

,Unnamed: 0,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,...,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Location,Year,TotalLaps,RaceProgress
0,0,0 days 00:08:53.241000,GAS,10,0 days 00:01:45.060000,1.0,NaN,0 days 00:00:04.103000,NaN,NaN,...,1.0,17.0,False,NaN,False,False,Melbourne,2018,58.0,0.017241
1,1,0 days 00:10:26.613000,GAS,10,0 days 00:01:33.372000,2.0,1.0,NaN,NaN,0 days 00:00:31.357000,...,1.0,17.0,False,NaN,False,True,Melbourne,2018,58.0,0.034483
2,2,0 days 00:11:59.474000,GAS,10,0 days 00:01:32.861000,3.0,1.0,NaN,NaN,0 days 00:00:31.160000,...,1.0,17.0,False,NaN,False,True,Melbourne,2018,58.0,0.051724
3,3,0 days 00:13:31.658000,GAS,10,0 days 00:01:32.184000,4.0,1.0,NaN,NaN,0 days 00:00:30.835000,...,1.0,17.0,False,NaN,False,True,Melbourne,2018,58.0,0.068966
4,4,0 days 00:15:03.990000,GAS,10,0 days 00:01:32.332000,5.0,1.0,NaN,NaN,0 days 00:00:30.716000,...,2.0,17.0,False,NaN,False,True,Melbourne,2018,58.0,0.086207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125754,125754,0 days 02:25:43.046000,SAR,2,0 days 00:01:21.280000,61.0,3.0,NaN,NaN,0 days 00:00:24.354000,...,1.0,20.0,False,NaN,False,True,Barcelona,2023,66.0,0.924242
125755,125755,0 days 02:27:05.180000,SAR,2,0 days 00:01:22.134000,62.0,3.0,NaN,NaN,0 days 00:00:23.675000,...,1.0,20.0,False,NaN,False,True,Barcelona,2023,66.0,0.939394
125756,125756,0 days 02:28:25.600000,SAR,2,0 days 00:01:20.420000,63.0,3.0,NaN,NaN,0 days 00:00:23.634000,...,1.0,20.0,False,NaN,False,True,Barcelona,2023,66.0,0.954545
125757,125757,0 days 02:29:45.580000,SAR,2,0 days 00:01:19.980000,64.0,3.0,NaN,NaN,0 days 00:00:23.602000,...,1.0,20.0,False,NaN,False,True,Barcelona,2023,66.0,0.969697
